# Week 5 - Vector Space Model (VSM) and Topic Modeling

Over the next weeks, we are going to re-implement Sherin's algorithm and apply it to the text data we've been working on last week! Here's our roadmap:

**Week 5 - data cleaning**
1. import the data
2. clean the data (e.g., remopve stop words, punctuation, etc.)
3. build a vocabulary for the dataset
4. create chunks of 100 words, with a 25-words overlap
5. create a word count matrix, where each chunk of a row and each column represents a word

**Week 6 - vectorization and linear algebra**
6. Dampen: weight the frequency of words (1 + log[count])
7. Scale: Normalize weighted frequency of words
8. Direction: compute deviation vectors

**Week 7 - Clustering**
9. apply different unsupervised machine learning algorithms
    * figure out how many clusters we want to keep
    * inspect the results of the clustering algorithm

**Week 8 - Visualizing the results**
10. create visualizations to compare documents

In [1]:
# in python code, our goal is to recreate the steps above as functions
# so that we can just one line to run topic modeling on a list of 
# documents: 
def ExtractTopicsVSM(documents, numTopics):
    ''' this functions takes in a list of documents (strings), 
        runs topic modeling (as implemented by Sherin, 2013)
        and returns the clustering results, the matrix used 
        for clustering a visualization '''
    
    # step 2: clean up the documents
    documents = clean_list_of_documents(documents)
    
    # step 3: let's build the vocabulary of these docs
    vocabulary = get_vocabulary(documents)
    
    # step 4: we build our list of 100-words overlapping fragments
    documents = flatten_and_overlap(documents)
    
    # step 5: we convert the chunks into a matrix
    matrix = docs_by_words_matrix(documents, vocabulary)
    
    # step 6: we weight the frequency of words (count = 1 + log(count))
    matrix = one_plus_log_mat(matrix, documents, vocabulary)
    
    # step 7: we normalize the matrix
    matrix = normalize(matrix)
    
    # step 8: we compute deviation vectors
    matrix = transform_deviation_vectors(matrix, documents)
    
    # step 9: we apply a clustering algorithm to find topics
    results_clustering = cluster_matrix(matrix)
    
    # step 10: we create a visualization of the topics
    visualization = visualize_clusters(results_clustering, vocabulary)
    
    # finally, we return the clustering results, the matrix, and a visualization
    return results_clustering, matrix, visualization

## Step 1 - Data Retrieval

In [59]:
# 1) using glob, find all the text files in the "Papers" folder
# Hint: refer to last week's notebook

import glob 
txt_files = glob.glob('papers/*.txt')
# print(txt_files)

In [6]:
# 2) get all the data from the text files into the "documents" list
# P.S. make sure you use the 'utf-8' encoding
# -*- coding: UTF-8 -*-
documents = []

for paper in txt_files:
    with open(paper) as filename: 
        contents = filename.read()
        documents.append(contents)
        
# print(documents[3])


111

towards closing the loop: bridging machine-induced
pedagogical policies to learning theories
guojing zhou, jianxun wang, collin f. lynch, min chi
department of computer science
north carolina state university
raleigh, nc 27695

{gzhou3,jwang75,cflynch,mchi}@ncsu.edu
abstract
in this study, we applied decision trees (dt) to extract
a compact set of pedagogical decision-making rules from
an original full set of 3,702 reinforcement learning (rl)induced rules, referred to as the dt-rl rules and full-rl
rules respectively. we then evaluated the effectiveness of
the two rule sets against a baseline random condition in
which the tutor made random yet reasonable decisions. we
explored two types of trees (weighted and unweighted) as
well as two pruning strategies (pre- and post-pruning). we
found that post-pruned weighted trees produced the best results with 529 dt-rl rules. the empirical evaluation was
conducted in a classroom study using an existing intelligent
tutoring system (its) nam

In [7]:
# 3) print the first 1000 characters of the first document to see what it 
# looks like (we'll use this as a sanity check below)

print (documents[0][0:1000])

103

epistemic network analysis and topic modeling for chat
data from collaborative learning environment
zhiqiang cai

brendan eagan

nia m. dowell

the university of memphis
365 innovation drive, suite 410
memphis, tn, usa

university of wisconsin-madison
1025 west johnson street
madison, wi, usa

the university of memphis
365 innovation drive, suite 410
memphis, tn, usa

zcai@memphis.edu

eaganb@gmail.com

niadowell@gmail.com

james w. pennebaker

david w. shaffer

arthur c. graesser

university of texas-austin
116 inner campus dr stop g6000
austin, tx, usa

university of wisconsin-madison
1025 west johnson street
madison, wi, usa

the university of memphis
365 innovation drive, suite 403
memphis, tn, usa

pennebaker@utexas.edu

dws@education.wisc.edu

art.graesser@gmail.com

abstract
this study investigates a possible way to analyze chat data from
collaborative learning environments using epistemic network
analysis and topic modeling. a 300-topic general topic model
built from tasa

## Step 2 - Data Cleaning

In [32]:
# 4) only select the text that's between the first occurence of the 
# the word "abstract" and the last occurence of the word "reference"
# Optional: print the length of the string before and after, as a 
# sanity check
# HINT: https://stackoverflow.com/questions/14496006/finding-last-occurrence-of-substring-in-string-replacing-that
# read more about rfind: https://www.tutorialspoint.com/python/string_rfind.htm

import re

documents_clean = []

for doc in documents:
    abstract = doc.find("abstract")
    ref = doc.rfind("reference")
    new_string = doc[abstract+8:ref] 
    #could have done length of abstract...or split the string after the len(abstract)...what's most efficient?
#     print(new_string)
#     print("doc "+ str(len(doc)))
#     print("new_string " + str(len(new_string)))
    documents_clean.append(new_string)




In [81]:
# 5) replace carriage returns (i.e., "\n") with a white space
# check that the result looks okay by printing the 
# first 1000 characters of the 1st doc:

documents_clean_white = []

for doc in documents_clean:
    doc = doc.replace("\n"," ")
    documents_clean_white.append(doc)
#     print(doc)

documents_clean_white[0][0:1000]

print(len(documents_clean_white))



17


In [39]:
# 6) replace the punctation below by a white space
# check that the result looks okay 
# (e.g., by print the first 1000 characters of the 1st doc)

punctuation = ['.', '...', '!', '#', '"', '%', '$', "'", '&', ')', 
               '(', '+', '*', '-', ',', '/', '.', ';', ':', '=', 
               '<', '?', '>', '@', '",', '".', '[', ']', '\\', ',',
               '_', '^', '`', '{', '}', '|', '~', '−', '”', '“', '’']

docs_clean_punct= []

for doc in documents_clean_white:
    for i in punctuation:
        doc = doc.replace(i, " ")
    docs_clean_punct.append(doc)

docs_clean_punct[0][0:1000]


' this study investigates a possible way to analyze chat data from collaborative learning environments using epistemic network analysis and topic modeling  a 300 topic general topic model built from tasa  touchstone applied science associates  corpus was used in this study  300 topic scores for each of the 15 670 utterances in our chat data were computed  seven relevant topics were selected based on the total document scores  while the aggregated topic scores had some power in predicting students  learning  using epistemic network analysis enables assessing the data from a different angle  the results showed that the topic score based epistemic networks between low gain students and high gain students were significantly different  𝑡   2 00   overall  the results suggest these two analytical approaches provide complementary information and afford new insights into the processes related to successful collaborative interactions   keywords chat  collaborative learning  topic modeling  epis

In [47]:
# 7) remove numbers by either a white space or the word "number"
# again, print the first 1000 characters of the first document
# to check that you're doing the right thing

docs_clean_num = []

for doc in docs_clean_punct:
    doc = re.sub('\w*[0-9]\w*', " ",doc)
    docs_clean_num.append(doc)
    
docs_clean_num[0][0:1000]



' this study investigates a possible way to analyze chat data from collaborative learning environments using epistemic network analysis and topic modeling  a   topic general topic model built from tasa  touchstone applied science associates  corpus was used in this study    topic scores for each of the     utterances in our chat data were computed  seven relevant topics were selected based on the total document scores  while the aggregated topic scores had some power in predicting students  learning  using epistemic network analysis enables assessing the data from a different angle  the results showed that the topic score based epistemic networks between low gain students and high gain students were significantly different  𝑡         overall  the results suggest these two analytical approaches provide complementary information and afford new insights into the processes related to successful collaborative interactions   keywords chat  collaborative learning  topic modeling  epistemic ne

In [82]:
# 8) Remove the stop words below from our documents
# print the first 1000 characters of the first document

# https://stackoverflow.com/questions/25346058/removing-list-of-words-from-a-string

stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 
              'ourselves', 'you', 'your', 'yours', 'yourself', 
              'yourselves', 'he', 'him', 'his', 'himself', 'she', 
              'her', 'hers', 'herself', 'it', 'its', 'itself', 
              'they', 'them', 'their', 'theirs', 'themselves', 
              'what', 'which', 'who', 'whom', 'this', 'that', 
              'these', 'those', 'am', 'is', 'are', 'was', 'were', 
              'be', 'been', 'being', 'have', 'has', 'had', 'having', 
              'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 
              'but', 'if', 'or', 'because', 'as', 'until', 'while', 
              'of', 'at', 'by', 'for', 'with', 'about', 'against', 
              'between', 'into', 'through', 'during', 'before', 
              'after', 'above', 'below', 'to', 'from', 'up', 'down', 
              'in', 'out', 'on', 'off', 'over', 'under', 'again', 
              'further', 'then', 'once', 'here', 'there', 'when', 
              'where', 'why', 'how', 'all', 'any', 'both', 'each', 
              'few', 'more', 'most', 'other', 'some', 'such', 'no', 
              'nor', 'not', 'only', 'own', 'same', 'so', 'than', 
              'too', 'very', 's', 't', 'can', 'will', 
              'just', 'don', 'should', 'now']

docs_clean_stop = []

for doc in docs_clean_num:
    docwords = doc.split()
    resultwords  = [word for word in docwords if word.lower() not in stop_words]
    result = ' '.join(resultwords)
    docs_clean_stop.append(result)

docs_clean_stop[0][0:1000]

print(len(docs_clean_stop))

17


In [53]:
# 9) remove words with one and two characters (e.g., 'd', 'er', etc.)
# print the first 1000 characters of the first document

docs_clean_short = []
shortword = re.compile(r'\W*\b\w{1,2}\b')

for doc in docs_clean_stop:
    doc = shortword.sub('',doc)
    docs_clean_short.append(doc)
    
docs_clean_short[0][0:1000]
    


'study investigates possible way analyze chat data collaborative learning environments using epistemic network analysis topic modeling topic general topic model built tasa touchstone applied science associates corpus used study topic scores utterances chat data computed seven relevant topics selected based total document scores aggregated topic scores power predicting students learning using epistemic network analysis enables assessing data different angle results showed topic score based epistemic networks low gain students high gain students significantly different overall results suggest two analytical approaches provide complementary information afford new insights processes related successful collaborative interactions keywords chat collaborative learning topic modeling epistemic network analysis introduction collaborative learning special form learning interaction affords opportunities groups students combine cognitive resources synchronously asynchronously participate tasks acco


### Putting it all together


In [92]:
# *** VERSION ONE***

# 10) package all of your work above into a function that cleans a given document

#store body of paper from abstract to reference
def paperbody(documents):
    docs_temp = []
    for doc in documents:
        abstract = doc.find("abstract")
        ref = doc.rfind("reference")
        new_string = doc[abstract+8:ref] 
        docs_temp.append(new_string)
#     print(docs_temp[0][0:1000])
    return docs_temp

# whitespace
def whitespace(documents):
    documents_clean_white = []
    for doc in documents_clean:
        doc = doc.replace("\n"," ")
        documents_clean_white.append(doc)
    return documents_clean_white

#punctuation
def punctuation(documents):
    punctuation = ['.', '...', '!', '#', '"', '%', '$', "'", '&', ')', 
               '(', '+', '*', '-', ',', '/', '.', ';', ':', '=', 
               '<', '?', '>', '@', '",', '".', '[', ']', '\\', ',',
               '_', '^', '`', '{', '}', '|', '~', '−', '”', '“', '’']
    docs_clean_punct= []

    for doc in documents_clean_white:
        for i in punctuation:
            doc = doc.replace(i, " ")
        docs_clean_punct.append(doc)
    return docs_clean_punct

# numbers
def numbers(documents):
    docs_clean_num = []
    for doc in docs_clean_punct:
        doc = re.sub('\w*[0-9]\w*', " ",doc)
        docs_clean_num.append(doc)
    return docs_clean_num

#stopwords
def stopwords(documents):
    stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 
              'ourselves', 'you', 'your', 'yours', 'yourself', 
              'yourselves', 'he', 'him', 'his', 'himself', 'she', 
              'her', 'hers', 'herself', 'it', 'its', 'itself', 
              'they', 'them', 'their', 'theirs', 'themselves', 
              'what', 'which', 'who', 'whom', 'this', 'that', 
              'these', 'those', 'am', 'is', 'are', 'was', 'were', 
              'be', 'been', 'being', 'have', 'has', 'had', 'having', 
              'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 
              'but', 'if', 'or', 'because', 'as', 'until', 'while', 
              'of', 'at', 'by', 'for', 'with', 'about', 'against', 
              'between', 'into', 'through', 'during', 'before', 
              'after', 'above', 'below', 'to', 'from', 'up', 'down', 
              'in', 'out', 'on', 'off', 'over', 'under', 'again', 
              'further', 'then', 'once', 'here', 'there', 'when', 
              'where', 'why', 'how', 'all', 'any', 'both', 'each', 
              'few', 'more', 'most', 'other', 'some', 'such', 'no', 
              'nor', 'not', 'only', 'own', 'same', 'so', 'than', 
              'too', 'very', 's', 't', 'can', 'will', 
              'just', 'don', 'should', 'now']
    docs_clean_stop = []
    for doc in docs_clean_num:
        docwords = doc.split()
        resultwords  = [word for word in docwords if word.lower() not in stop_words]
        result = ' '.join(resultwords)
        docs_clean_stop.append(result)
    return docs_clean_stop

def shortwords(documents):
    docs_clean_short = []
    shortword = re.compile(r'\W*\b\w{1,2}\b')
    for doc in docs_clean_stop:
        doc = shortword.sub('',doc)
        docs_clean_short.append(doc)
    return docs_clean_short

def clean_list_of_documents(documents):
    
    cleaned_docs = []

    ### your code ###

    cleaned_docs = paperbody(documents)
    cleaned_docs = whitespace(documents)
    cleaned_docs = punctuation(documents)
    cleaned_docs = numbers(documents)
    cleaned_docs = stopwords(documents)
    cleaned_docs = shortwords(documents)
    
    return cleaned_docs



In [100]:
# *** Version TWO***
# Yes I know this is shorter but I wrote the first version first

def clean_list_of_documents2(documents):    
    cleaned_docs = []
    stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 
              'ourselves', 'you', 'your', 'yours', 'yourself', 
              'yourselves', 'he', 'him', 'his', 'himself', 'she', 
              'her', 'hers', 'herself', 'it', 'its', 'itself', 
              'they', 'them', 'their', 'theirs', 'themselves', 
              'what', 'which', 'who', 'whom', 'this', 'that', 
              'these', 'those', 'am', 'is', 'are', 'was', 'were', 
              'be', 'been', 'being', 'have', 'has', 'had', 'having', 
              'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 
              'but', 'if', 'or', 'because', 'as', 'until', 'while', 
              'of', 'at', 'by', 'for', 'with', 'about', 'against', 
              'between', 'into', 'through', 'during', 'before', 
              'after', 'above', 'below', 'to', 'from', 'up', 'down', 
              'in', 'out', 'on', 'off', 'over', 'under', 'again', 
              'further', 'then', 'once', 'here', 'there', 'when', 
              'where', 'why', 'how', 'all', 'any', 'both', 'each', 
              'few', 'more', 'most', 'other', 'some', 'such', 'no', 
              'nor', 'not', 'only', 'own', 'same', 'so', 'than', 
              'too', 'very', 's', 't', 'can', 'will', 
              'just', 'don', 'should', 'now']
    punctuation = ['.', '...', '!', '#', '"', '%', '$', "'", '&', ')', 
               '(', '+', '*', '-', ',', '/', '.', ';', ':', '=', 
               '<', '?', '>', '@', '",', '".', '[', ']', '\\', ',',
               '_', '^', '`', '{', '}', '|', '~', '−', '”', '“', '’']
    shortword = re.compile(r'\W*\b\w{1,2}\b')
    for doc in documents:
        abstract = doc.find("abstract")
        ref = doc.rfind("reference")
        new_string = doc[abstract+8:ref] 
        doc = new_string.replace("\n"," ")
        for i in punctuation:
            doc = doc.replace(i, " ")
        doc = re.sub('\w*[0-9]\w*', " ",doc)
        docwords = doc.split()
        resultwords  = [word for word in docwords if word.lower() not in stop_words]
        result = ' '.join(resultwords)   
        doc = shortword.sub('',result)
        cleaned_docs.append(doc)
    return cleaned_docs




In [101]:
# 11a) reimport your raw data using the code in 2)
documents = []

for paper in txt_files:
    with open(paper) as filename: 
        contents = filename.read()
        documents.append(contents)

        
# 11b) clean your files using the function above
# clean_docs = clean_list_of_documents(documents)

clean_docs2 = clean_list_of_documents2(documents)


# 11c) print the first 1000 characters of the first document

# print(clean_docs[0][:1000])

print(clean_docs2[0][:1000])

study investigates possible way analyze chat data collaborative learning environments using epistemic network analysis topic modeling topic general topic model built tasa touchstone applied science associates corpus used study topic scores utterances chat data computed seven relevant topics selected based total document scores aggregated topic scores power predicting students learning using epistemic network analysis enables assessing data different angle results showed topic score based epistemic networks low gain students high gain students significantly different overall results suggest two analytical approaches provide complementary information afford new insights processes related successful collaborative interactions keywords chat collaborative learning topic modeling epistemic network analysis introduction collaborative learning special form learning interaction affords opportunities groups students combine cognitive resources synchronously asynchronously participate tasks accom

## Step 3 - Build your list of vocabulary

This list of words (i.e., the vocabulary) is going to become the columns of your matrix.

In [103]:
import math
import numpy as np

12) Describe why we need to figure out the vocabulary used in our corpus (refer back to Sherin's paper, and explain in your own words): 

In [111]:
# 13) create a function that takes in a list of documents
# and returns a set of unique words. Make sure that you
# sort the list alphabetically before returning it. 

def get_vocabulary(documents):
    voc = []
    for doc in documents:
        docList = doc.split()
        for word in docList:
            if word not in voc:
                voc.append(word)
    return voc


documents = []

for paper in txt_files:
    with open(paper) as filename: 
        contents = filename.read()
        documents.append(contents)
        
vocabulary = get_vocabulary(clean_docs2)

print(len(vocabulary))

# print(vocabulary)

# Then print the length of your vocabulary (it should be 
# around 5500 words)

# ^^ assuming the cleaned dictionary


5639


In [112]:
# 14) what was the size of Sherin's vocabulary? 

# "For the corpus used in this work, the full vocabulary contained
# 1,429 words, the stop list consisted of 782 words, and the resulting pruned vocabulary
# contained 647 words." (p.617)

## Step 4 - transform your documents into 100-words chunks

In [132]:
# 15) create a function that takes in a list of documents
# and returns a list of 100-words chunk 
# (with a 25 words overlap between them)
# Optional: add two arguments, one for the number of words
# in each chunk, and one for the overlap size
# Advice: combining all the documents into one giant string
# and splitting it into separate words will make your life easier!


docs_all = ''.join(clean_docs2)

# print(docs_all)

def wordChunk (listName, chunkSize, overlap):
    HundList = []
    docsList = listName.split()
    for i in range(0, len(docsList), chunkSize-overlap):
#         print(i)
        HundList.append(" ".join(docsList[i:i+chunkSize]))
#         print(i+chunkSize)
    return HundList

docs_all = wordChunk(docs_all, 100, 25)

# print(docs_all[0])
# print(docs_all[1])
# print(docs_all[2])


In [140]:
# 16) create a for loop to double check that each chunk has 
# a length of 100
# Optional: use assert to do this check

for chunk in docs_all:
    chunk = chunk.split()
#     print (len(chunk))
#     assert len(chunk) == True
#     print (len(chunk))


In [141]:
# 17) print the first chunk, and compare it to the original text.
# does that match what Sherin describes in his paper?

print(docs_all[0])

study investigates possible way analyze chat data collaborative learning environments using epistemic network analysis topic modeling topic general topic model built tasa touchstone applied science associates corpus used study topic scores utterances chat data computed seven relevant topics selected based total document scores aggregated topic scores power predicting students learning using epistemic network analysis enables assessing data different angle results showed topic score based epistemic networks low gain students high gain students significantly different overall results suggest two analytical approaches provide complementary information afford new insights processes related successful collaborative interactions keywords chat collaborative learning topic modeling epistemic network analysis


In [ ]:
# 18) how many chunks did Sherin have? What does a chunk become 
# in the next step of our topic modeling algorithm? 


In [23]:
# 19) what are some other preprocessing steps we could do 
# to improve the quality of the text data? Mention at least 2.


In [25]:
# 20) in your own words, describe the next steps of the 
# data modeling algorithms (listed below):


## Step 5 - Vector and Matrix operations

## Step 6 - Weight word frequency

## Step 7 - Matrix normalization

## Step 8 - Deviation Vectors

## Step 9 - Clustering

## Step 10 - Visualizing the results

## Final Step - Putting it all together: 

In [17]:
# in python code, our goal is to recreate the steps above as functions
# so that we can just one line to run topic modeling on a list of 
# documents: 
def ExtractTopicsVSM(documents, numTopics):
    ''' this functions takes in a list of documents (strings), 
        runs topic modeling (as implemented by Sherin, 2013)
        and returns the clustering results, the matrix used 
        for clustering a visualization '''
    
    # step 2: clean up the documents
    documents = clean_list_of_documents(documents)
    
    # step 3: let's build the vocabulary of these docs
    vocabulary = get_vocabulary(documents)
    
    # step 4: we build our list of 100-words overlapping fragments
    documents = flatten_and_overlap(documents)
    
    # step 5: we convert the chunks into a matrix
    matrix = docs_by_words_matrix(documents, vocabulary)
    
    # step 6: we weight the frequency of words (count = 1 + log(count))
    matrix = one_plus_log_mat(matrix, documents, vocabulary)
    
    # step 7: we normalize the matrix
    matrix = normalize(matrix)
    
    # step 8: we compute deviation vectors
    matrix = transform_deviation_vectors(matrix, documents)
    
    # step 9: we apply a clustering algorithm to find topics
    results_clustering = cluster_matrix(matrix)
    
    # step 10: we create a visualization of the topics
    visualization = visualize_clusters(results_clustering, vocabulary)
    
    # finally, we return the clustering results, the matrix, and a visualization
    return results_clustering, matrix, visualization